In [1]:
from keras import backend as K
K.set_learning_phase(0)

Using TensorFlow backend.


In [2]:
from keras.models import load_model
model = load_model('ResNet50.h5')

c:\users\asirra\appdata\local\programs\python\python35\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
model.input

<tf.Tensor 'input_1:0' shape=(?, 1080, 1920, 3) dtype=float32>

In [5]:
model.output

<tf.Tensor 'fc1000/Softmax:0' shape=(?, 1000) dtype=float32>

In [3]:
import tensorflow as tf

def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        frozen_graph = tf.compat.v1.graph_util.remove_training_nodes(frozen_graph, protected_nodes=[output_names])
        return frozen_graph

In [6]:
frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])

INFO:tensorflow:Froze 532 variables.
INFO:tensorflow:Converted 532 variables to const ops.


In [9]:
tf.train.write_graph(frozen_graph, "./", "ResNet50HD_tf_model.pb", as_text=False)

'./ResNet50HD_tf_model.pb'